# **Imports &#8595;**

In [5]:
# !pip uninstall -q -y accelerate transformers torch 
# !pip uninstall -q -y sentence-transformers
# !pip install -q accelerate==0.21.0
# !pip install -q transformers==4.24.0
# !pip install -q pytorch
# !pip install sentence-transformers==2.2.2
# !pip show accelerate

In [6]:
import numpy as np
import pandas as pd 
import re
i
# import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from sentence_transformers import InputExample
from torch.utils.data import DataLoader
from sentence_transformers import models, SentenceTransformer
from sentence_transformers.losses import DenoisingAutoEncoderLoss
from sentence_transformers.datasets import SentencesDataset

# **Load Dataset &#8595;**

In [7]:
DATA_PATH = "datasets/eedi-mining-misconceptions-in-mathematics"
EXTERNAL_DATA_PATH = "datasets/eedi-external-dataset"

# train_df = pd.read_csv(f'{DATA_PATH}/train.csv', index_col='QuestionId')
train_df = pd.read_csv(f'{EXTERNAL_DATA_PATH}/all_train.csv', index_col='QuestionId') #this contains the original dataset + an external dataset generated by a LLM
test_df = pd.read_csv(f'{DATA_PATH}/test.csv')
misconceptions_df = pd.read_csv(f'{DATA_PATH}/misconception_mapping.csv')

pd.options.display.max_colwidth = 300
display(train_df.head(5))
pd.options.display.max_colwidth = 50

,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId,source,MisconceptionAName,MisconceptionBName,MisconceptionCName,MisconceptionDName,OriginalQuestionId
QuestionId,,,,,,,,,,,,,,,,,,,,
0,856.0,Use the order of operations to carry out calculations involving powers,33.0,BIDMAS,A,\[\r\n3 \times 2+4-5\r\n\]\r\nWhere do the brackets need to go to make the answer equal \( 13 \) ?,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0,original,NaN,NaN,NaN,"Confuses the order of operations, believes addition comes before multiplication",NaN
1,1612.0,Simplify an algebraic fraction by factorising the numerator,1077.0,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{m^{2}+2 m-3}{m-3} \)",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN,original,"Does not know that to factorise a quadratic expression, to find two numbers that add to give the coefficient of the x term, and multiply to give the non variable term\r\n","Thinks that when you cancel identical terms from the numerator and denominator, they just disappear","Does not know that to factorise a quadratic expression, to find two numbers that add to give the coefficient of the x term, and multiply to give the non variable term\r\n",NaN,NaN
2,2774.0,Calculate the range from a list of data,339.0,Range and Interquartile Range from a List of Data,B,"Tom and Katie are discussing the \( 5 \) plants with these heights:\r\n\( 24 \mathrm{~cm}, 17 \mathrm{~cm}, 42 \mathrm{~cm}, 26 \mathrm{~cm}, 13 \mathrm{~cm} \)\r\nTom says if all the plants were cut in half, the range wouldn't change.\r\nKatie says if all the plants grew by \( 3 \mathrm{~cm} \)...",Only\r\nTom,Only\r\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0,original,Believes if you changed all values by the same proportion the range would not change,NaN,Believes if you changed all values by the same proportion the range would not change,Believes if you add the same value to all numbers in the dataset the range will change,NaN
3,2377.0,Recall and use the intersecting diagonals properties of a rectangle,88.0,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with different length sides can never be... ![A rectangle with the diagonals drawn in. The angle on the right hand side at the centre is highlighted in red and the angle at the bottom at the centre is highlighted in yellow.](),acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0,original,Does not know the properties of a rectangle,Does not know the properties of a rectangle,NaN,Does not know the properties of a rectangle,NaN
4,3387.0,Substitute positive integer values into formulae involving powers or roots,67.0,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find values in the table below. What is the value covered by the star? \begin{tabular}{|c|c|c|c|c|}\r\n\hline\( r \) & \( 1 \) & \( 2 \) & \( 3 \) & \( 4 \) \\\r\n\hline\( f \) & \( 6 \) & \( 15 \) & \( \color{gold}\bigstar \) & \\\r\n\hline\r\n\end{tabu...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0,original,NaN,NaN,NaN,Thinks you can find missing values in a given table by treating the row as linear and adding on the difference between the first two values given.,NaN


# **Data Preprocessing &#8595;**

In [8]:
def clean(example, columns):
    """
    Cleans the example from the Dataset
    Args:
        example: an example from the Dataset
        columns: columns that will be cleaned

    Returns: update example containing 'clean' columns

    """
    for col in columns:
        text = example[f'{col}']

        # Empty text
        if type(text) not in (str, np.str_) or text=='':
            example[f'clean_{col}'] = ''
            return example

        # 'text' from the example can be of type numpy.str_, let's convert it to a python str
        text = str(text).lower()

        # Clean the text
        text = re.sub("\"", " ", text) # removes the " from certain texts
        text = re.sub("\n", " ", text) # removes the multiple "\n" 
        text = re.sub(r"(\\\w+)(\W)", r" \1 \2", text) # matches with the LaTeX commands like "\hline{}",... and transforms them to " \hline {}"
        text = re.sub(r"([\(|\{|\[|\|])", r" \1", text) # matches every opening parenthesis types and puts spaces on their left
        text = re.sub(r"([\)|\}|\]])", r"\1 ", text) # matches every closing parenthesis types and puts spaces on their right
        text = re.sub(r"\\(?![a-zA-Z])", " ", text) # removes every backslash that is not the start of a LaTeX command
        text = re.sub(r"\( | \)", "", text) # removes the parentheses that appear sometimes from nowhere 
        text = re.sub(r"\[ | \]", "", text) # removes the parentheses that appear sometimes from nowhere
        
        text = re.sub(r" +", " ", text) # cleans the double spaces made by above substitutions
        # Update the example with the cleaned text
        example[f'clean_{col}'] = text.strip()
    return example

# testing_data = {
#     'QuestionText': ["This is a question with a newline\nin the middle"],
#     'AnswerAText': ["Answer A\nwith newline and \\table[test]"],
#     'AnswerBText': ["Answer B\nwith newline and \hline(uwo)"],
#     'AnswerCText': ["Answer C\nwith newline and \color{gold}"],
#     'AnswerDText': ["Answer D\nwith newline and \\begin{tabular}"]
# }
# df = pd.DataFrame(testing_data)
# df = df.apply(clean, axis = 1, columns = columns_to_clean)
# display(df.head(1))

columns_to_clean = ['QuestionText', 'AnswerAText', 'AnswerBText', 'AnswerCText', 'AnswerDText']
train_df = train_df.apply(clean, axis = 1, columns = columns_to_clean)

# Adjust column order
new_order = ['ConstructId', 'ConstructName', 'SubjectId', 'SubjectName', 'CorrectAnswer']
for col in columns_to_clean:
    new_order.append(col)
    new_order.append(f'clean_{col}')
new_order.extend(['MisconceptionAId', 'MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId'])
train_df = train_df[new_order]


display_train_df = train_df[['QuestionText', 'clean_QuestionText','AnswerAText', 'clean_AnswerAText', 'AnswerBText', 'clean_AnswerBText', 'AnswerCText', 'clean_AnswerCText', 'AnswerDText', 'clean_AnswerDText']]
pd.options.display.max_colwidth = 300
display(display_train_df.head(1))
pd.options.display.max_colwidth = 50

display(misconceptions_df.head(1))

,QuestionText,clean_QuestionText,AnswerAText,clean_AnswerAText,AnswerBText,clean_AnswerBText,AnswerCText,clean_AnswerCText,AnswerDText,clean_AnswerDText
QuestionId,,,,,,,,,,
0,\[\r\n3 \times 2+4-5\r\n\]\r\nWhere do the brackets need to go to make the answer equal \( 13 \) ?,[\r 3 \times 2+4-5\r \r where do the brackets need to go to make the answer equal 13 ?,\( 3 \times(2+4)-5 \),3 \times (2+4) -5,\( 3 \times 2+(4-5) \),3 \times 2+ (4-5),\( 3 \times(2+4-5) \),3 \times (2+4-5),Does not need brackets,does not need brackets


,MisconceptionId,MisconceptionName
0,0,Does not know that angles in a triangle sum to...


# **Reshape Dataset For Training &#8595;**

In [9]:
# train_df columns: QuestionID, ConstructID, ConstructName, CorrectAnswer, SubjectId, SubjectName, QuestionText, Answer[A/B/C/D]Text, Misconception[A/B/C/D]Id

reshaped_data = []
for _, row in train_df.iterrows():
    for answer, misconception_id in zip(
        ['clean_AnswerAText', 'clean_AnswerBText', 'clean_AnswerCText', 'clean_AnswerDText'],
        ['MisconceptionAId', 'MisconceptionBId', 'MisconceptionCId', 'MisconceptionDId']
    ): # turn the data into a format where each datapoint (row) represents an answer choice (i.e there are now 4 datapoints for each question)
        misc_id = int(row[misconception_id]) if not pd.isna(row[misconception_id]) else row[misconception_id]
        reshaped_data.append({
            'QuestionText': row['clean_QuestionText'],
            'AnswerText': row[answer],
            'MisconceptionId': misc_id,
            'MisconceptionText': misconceptions_df.loc[misconceptions_df['MisconceptionId'] == misc_id, 'MisconceptionName'].values[0] if not pd.isna(misc_id) else misc_id,
            'SubjectName': row['SubjectName'],
            'ConstructName': row['ConstructName']
        })

reshaped_df = pd.DataFrame(reshaped_data)
display(reshaped_df.head())

# removed columns: QuestionId, ConstructId, CorrectAnswer, SubjectId
# other changes: Answer[A/B/C/D]Text are now in separate datapoints along with their associated Misconception[A/B/C/D]Texts 

,QuestionText,AnswerText,MisconceptionId,MisconceptionText,SubjectName,ConstructName
0,[\r 3 \times 2+4-5\r \r where do the brackets ...,3 \times (2+4) -5,NaN,NaN,BIDMAS,Use the order of operations to carry out calcu...
1,[\r 3 \times 2+4-5\r \r where do the brackets ...,3 \times 2+ (4-5),NaN,NaN,BIDMAS,Use the order of operations to carry out calcu...
2,[\r 3 \times 2+4-5\r \r where do the brackets ...,3 \times (2+4-5),NaN,NaN,BIDMAS,Use the order of operations to carry out calcu...
3,[\r 3 \times 2+4-5\r \r where do the brackets ...,does not need brackets,1672.0,"Confuses the order of operations, believes add...",BIDMAS,Use the order of operations to carry out calcu...
4,"simplify the following, if possible: \frac {m^...",m+1,2142.0,Does not know that to factorise a quadratic ex...,Simplifying Algebraic Fractions,Simplify an algebraic fraction by factorising ...


# **Sentence Embeddings & OneHot Encoding&#8595;**

In [10]:
# remove NaN values (dropping all datapoints that do not have misconceptions assigned to them)
# P.S. that means we are also deleting all the rows (answer choices) that are correct
# P.P.S. unless somehow there are correct answers that have misconceptions associated with them
print(reshaped_df['MisconceptionId'].isnull().sum())  # 10582 NaN values yikes :/
reshaped_df = reshaped_df.dropna(subset=['MisconceptionId'])
print(reshaped_df['MisconceptionId'].isnull().sum())  # 0 now yippie


10582
0


In [11]:
print(reshaped_df.shape)

(4370, 6)


In [28]:
# generate sentence embeddings
reshaped_df['CombinedText'] = reshaped_df['QuestionText'] + " " + reshaped_df['AnswerText']

# model_name = 'sentence-transformers/all-MiniLM-L6-v2'
# model_name = 'sentence-transformers/all-MiniLM-L12-v2'
# model_name = 'hkunlp/instructor-large'
model_name = "Qwen/Qwen2.5-0.5B-Instruct"  
# model_name = 'math-similarity/Bert-MLM_arXiv-MP-class_zbMath'
# model = SentenceTransformer(model_name)

fine_tuned_model = False

def embed_text(text, model):
    return model.encode(text, convert_to_tensor=True)

output_path = "output/tsdae-model-qwen"

In [23]:
if model_name != 'hkunlp/instructor-large' and fine_tuned_model:
    word_embedding_model = models.Transformer(model_name)
    pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
    tsdae_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
    train_loss = DenoisingAutoEncoderLoss(tsdae_model, tie_encoder_decoder=True)

    train_examples = [
        InputExample(texts=[row['CombinedText'], row['MisconceptionText']])
        for _, row in reshaped_df.iterrows()
    ]

    train_dataset = SentencesDataset(train_examples, model=None)  
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=8)

    print("Starting training")
    tsdae_model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1, 
        warmup_steps=reshaped_df.shape[1] * 0.15,  
        optimizer_params={'lr': 1e-5},  
        output_path=output_path
    )

Starting training


 91%|█████████▏| 500/547 [3:56:31<32:33, 41.56s/it]  

{'loss': 1.4995, 'grad_norm': 46.11640167236328, 'learning_rate': 8.441141389118268e-07, 'epoch': 0.91}


100%|██████████| 547/547 [4:17:51<00:00, 28.29s/it]


{'train_runtime': 15471.8546, 'train_samples_per_second': 0.282, 'train_steps_per_second': 0.035, 'train_loss': 1.4569348504382034, 'epoch': 1.0}


In [29]:
# use One hot encoding for categorical data (create a "column" for each unique subject and construct and represent each row with 0 and 1)
# model = SentenceTransformer(model_name)

if fine_tuned_model:
    model = SentenceTransformer(output_path)
    X_embeddings = np.array([embed_text(text, model) for text in reshaped_df['CombinedText']])
else:
    model = SentenceTransformer(model_name)
    X_embeddings = np.array([embed_text(text, model) for text in reshaped_df['CombinedText']])

encoder = OneHotEncoder(sparse_output=False)
categorical_features = encoder.fit_transform(reshaped_df[['SubjectName', 'ConstructName']])

No sentence-transformers model found with name Qwen/Qwen2.5-0.5B-Instruct. Creating a new one with mean pooling.


In [30]:
# display(X_embeddings.shape)
# X_embeddings = X_embeddings.squeeze(axis=1)
# display(X_embeddings.shape)

X = np.hstack([X_embeddings, categorical_features])
y = reshaped_df['MisconceptionText']

print(X.shape) # print shape of X to see total number of features 

# random_state = np.random.randint(0, 10000)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42) #Split the data into training and testing
# print(f'random_state: {random_state}')

(4370, 1816)


# **Random Forest Training&#8595;**

In [31]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

# y_pred = rf_classifier.predict(X)
# print(classification_report(y_test, y_pred))

RandomForestClassifier(random_state=42)

# **Testing&#8595;**

In [20]:
def map_at_25(y_true, y_pred_probs, top_k=25):
    
    map_25 = 0.0
    for true_label, pred_prob in zip(y_true, y_pred_probs):
        # Get top_k predictions
        top_preds = np.argsort(pred_prob)[::-1][:top_k]
        
        if not true_label:
            continue
        
        score = 0.0
        hits = 0
        for i, pred in enumerate(top_preds, start=1):
            if pred == true_label:
                hits += 1
                score += hits / i  # Precision at i
        
        # Average Precision at 25
        map_25 += score / min(1, top_k)
    
    return map_25 / len(y_true)

In [32]:
y_val_pred_probs = rf_classifier.predict_proba(X_val)  
y_val_true = list(y_val)
y_val_true_id = []
for val in y_val_true:
    matching_row = misconceptions_df[misconceptions_df['MisconceptionName'] == val]
    if not matching_row.empty:  # Check if a matching row exists
        misconception_id = matching_row.iloc[0]['MisconceptionId']  # Get the MisconceptionId
        y_val_true_id.append(misconception_id)
    else:
        y_val_true_id.append(None)

map25_score = map_at_25(y_val_true_id, y_val_pred_probs)

print(f"MAP@25 Score: {map25_score}")


MAP@25 Score: 0.0057335367404017295


In [13]:
y_pred_probs = rf_classifier.predict_proba(X_val)

# print predictions
for idx, (true_label, pred_prob) in enumerate(zip(y_val, y_pred_probs)):
    # Get top 25 predictions and probabilities
    top_preds = np.argsort(pred_prob)[::-1][:25]
    top_probs = pred_prob[top_preds]
    
    # Check if true is within top 25
    in_top_25 = true_label in top_preds
    
    print(f"Example {idx + 1}")
    print(f"True Label: {true_label}")
    print("Top 25 Predictions (Misconception ID: Probability):")
    for pred, prob in zip(top_preds, top_probs):
        print(f"ID {pred}: {prob:.4f}")
    print(f"True Label in Top 25: {in_top_25}\n")

    # Number of questions to print
    if idx == 10:  
        break

Example 1
True Label: 2030.0
Top 25 Predictions (Misconception ID: Probability):
ID 1052: 0.2400
ID 121: 0.0400
ID 494: 0.0400
ID 1138: 0.0300
ID 1025: 0.0300
ID 822: 0.0200
ID 70: 0.0200
ID 495: 0.0200
ID 100: 0.0200
ID 942: 0.0200
ID 95: 0.0100
ID 1339: 0.0100
ID 631: 0.0100
ID 98: 0.0100
ID 297: 0.0100
ID 989: 0.0100
ID 629: 0.0100
ID 1437: 0.0100
ID 79: 0.0100
ID 78: 0.0100
ID 1117: 0.0100
ID 416: 0.0100
ID 61: 0.0100
ID 762: 0.0100
ID 761: 0.0100
True Label in Top 25: False

Example 2
True Label: 1794.0
Top 25 Predictions (Misconception ID: Probability):
ID 693: 0.8000
ID 96: 0.1000
ID 404: 0.0500
ID 69: 0.0300
ID 1023: 0.0100
ID 836: 0.0100
ID 480: 0.0000
ID 473: 0.0000
ID 474: 0.0000
ID 475: 0.0000
ID 476: 0.0000
ID 477: 0.0000
ID 478: 0.0000
ID 479: 0.0000
ID 482: 0.0000
ID 481: 0.0000
ID 483: 0.0000
ID 484: 0.0000
ID 485: 0.0000
ID 486: 0.0000
ID 487: 0.0000
ID 488: 0.0000
ID 489: 0.0000
ID 472: 0.0000
ID 1437: 0.0000
True Label in Top 25: False

Example 3
True Label: 649.0
To